In [328]:
from skimage import io
from skimage import morphology
from skimage import util
from skimage import measure
from skimage import draw
from skimage import color
from skimage import transform
from skimage import exposure
from skimage import filters

from collections import Counter
from pathlib import Path


import numpy as np
%matplotlib qt 

# Aktualne

### Pomocnicze

In [386]:
def remove_outliers_centroids(data, quantile_height=0.9, quantile_width=0.9):
    Q1 = np.quantile(data[:, 0], 1-quantile_height)
    Q3 = np.quantile(data[:, 0], quantile_height)
    IQR = Q3 - Q1
    data = np.array([el for el in data if el[0] > Q1 and el[0] < Q3])

    Q1 = np.quantile(data[:, 1], 1-quantile_width)
    Q3 = np.quantile(data[:, 1], quantile_width)
    IQR = Q3 - Q1
    data = np.array([el for el in data if el[1] > Q1 and el[1] < Q3])
    
    return data

# def remove_background_in_parts(img, rows=2, columns=2):
#     height, width = img.shape
    
#     height_part = int(height/rows)
#     width_part = int(width/columns)
      
            
#     for row in range(rows):
#         for col in range(columns):
#             sector = img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part]
#             img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part] = remove_background_rectangle(sector)
            
#     return img


def remove_background_on_left_side(img):
    img_org = img.copy()
    
    p2, p98 = np.percentile(img, (2, 98))
    img = exposure.rescale_intensity(img_slice, in_range=(p2, p98))
    # Sprawdzamy średnią jasność obrazu. Następnie bierzemy po kolei grupy kolumn (skłądające się z step kolumn).
    # Jeżeli jasność w grupie jest mniejsza niż (średnia w obrazie - bias) to zastępujemy wartością średnią.
    mean_value_in_img = np.mean(img) 
    step = 10
    bias = 0.2 # 0.07 bez roziągania histogramu
    was_action = True 
    for el in range(0, len(img.T), step):
        if np.mean(img[:,el:el+step]) < mean_value_in_img-bias:
#             img[:,el:el+step] = mean_value_in_img
            img_org = img_org[:,el+step:]
            was_action = True
        else:
            was_action = False
        
        if not was_action:
#             img[:,el:el+step] = mean_value_in_img
            img_org = img_org[:,el+int(step/2):]
            break
            
    return img_org

def remove_background(img):
     # usuwa z lewej
    img = remove_background_on_left_side(img)
    
     # usuwa z prawej
    img = transform.rotate(img, 180)
    img = remove_background_on_left_side(img)
    
    # usuwa z dołu
    img = img.T
    img = remove_background_on_left_side(img)
    
    # usuwa z góry
    img = transform.rotate(img, 180)
    img = remove_background_on_left_side(img)
    
    # wraca do oryginalnej postaci
    img = img.T
    
    return img


### Główny program

In [388]:
source_path = Path('../data/partial_results/canny_adam')
save_path = Path('../data/partial_results/wyciete_fragmenty')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.png")
for image_path in images_paths:
#     image_path = Path("..\data\partial_results\canny_adam\img_17.png")
    print(image_path)
    img = io.imread(image_path)
      
    # Szukamy regionów. Więkoszość z nich powinna znajdować się w obszarze tekstu.
    label_img = measure.label(img)
    regions = measure.regionprops(label_img)
    regions_centroids = np.array([reg.centroid for reg in regions])
    mean_centroid = (np.mean(regions_centroids[:, 0]), np.mean(regions_centroids[:, 1]))

    # Usuwamy obszary, który centoridy zbyt mocno odstają. Dwukrotnie.
    data = remove_outliers_centroids(regions_centroids, quantile_height=0.95, quantile_width=0.9)
    data = remove_outliers_centroids(data, quantile_height=0.95, quantile_width=0.95)

    # Z pozostałych centoridów tworzymy prostokąt troche powiększony.
    height_min = np.min(data[:, 0])
    width_min = np.min(data[:, 1])
    height_max = np.max(data[:, 0])
    width_max = np.max(data[:, 1])
    
    img_height, img_width = img.shape
    
    start_point_height = int(max(height_min-img_height*0.09, 1))
    start_point_width = int(max(width_min-img_width*0.15, 1))
    end_point_height = int(height_max+img_height*0.1)
    end_point_width = int(width_max+img_width*0.25)
    
    # Wycięcie tego prostokątu z oryginalnego obrazu.
    img_org_path = Path('../data/ocr1') / str(image_path.stem + ".jpg")
    img_org = io.imread(img_org_path)
#     img_org = color.rgb2gray(img_org)    
    img_slice = img_org[start_point_height:end_point_height, start_point_width:end_point_width]
#     io.imsave(arr=util.img_as_ubyte(img_slice), fname=save_path / (image_path.stem+'.png'))
    
    # Na wycinkach nadal czasami pojawia się stół. Więc usuwamy te fragmenty.
    img_removed_background = remove_background(img_slice)

    save_path_removed_background = save_path / "usuniety_stol"
    save_path_removed_background.mkdir(parents=True, exist_ok=True)
    io.imsave(arr=util.img_as_ubyte(img_removed_background), fname=save_path_removed_background / (image_path.stem+'.png'))

    
    
    # Kolorowanie do testów
#     img = color.gray2rgb(img)
#     rr, cc = draw.rectangle_perimeter((max(height_min-img_height*0.075, 1), max(width_min-img_width*0.15, 1)), 
#                                       (height_max+img_height*0.1, width_max+img_width*0.25))
#     img[rr, cc] = (255,0,0)
    
#     for cent in data:
#         rr, cc = draw.circle(r=cent[0], c=cent[1], radius=10)
#         img[rr, cc] = (255,0,0)
    

#     io.imsave(arr=util.img_as_ubyte(img), fname=save_path / (image_path.stem+'.png'))
        
    


..\data\partial_results\canny_adam\img_17.png


# V2 - wycinanie kwadratów a nie kolumn do usunięcia tła - NIE DZIAŁA

In [244]:
def remove_background_rectangle(img, rows=40, columns=40):
    height, width = img.shape
    mean_value_in_img = np.mean(img) 
    step=50
    bias = 0.07
    
    height_part = int(height/rows)
    width_part = int(width/columns)
    
    
    for row in range(0, height, step):
        for col in range(0, width, step):
            sector = img[row:row+step,col:col+step]
            mean_value_in_sector = np.mean(sector)
            if mean_value_in_sector < mean_value_in_img-bias:
                img[row:row+step,col:col+step] = mean_value_in_img
            
#     for row in range(rows):
#         for col in range(columns):
#             sector = img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part]
#             mean_value_in_sector = np.mean(sector)
#             if mean_value_in_sector < mean_value_in_img-bias:
#                 img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part] = mean_value_in_img
            
    return img

In [245]:
source_path = Path('../data/partial_results/canny_adam')
save_path = Path('../data/partial_results/wyciete_fragmenty')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.png")
for image_path in images_paths:
#     image_path = Path("..\data\partial_results\canny_adam\img_3.png")
    print(image_path)
    img = io.imread(image_path)
      
    # Szukamy regionów. Więkoszość z nich powinna znajdować się w obszarze tekstu.
    label_img = measure.label(img)
    regions = measure.regionprops(label_img)
    regions_centroids = np.array([reg.centroid for reg in regions])
    mean_centroid = (np.mean(regions_centroids[:, 0]), np.mean(regions_centroids[:, 1]))

    # Usuwamy obszary, który centoridy zbyt mocno odstają. Dwukrotnie.
    data = remove_outliers_centroids(regions_centroids, quantile_height=0.95, quantile_width=0.9)
    data = remove_outliers_centroids(data, quantile_height=0.95, quantile_width=0.95)

    # Z pozostałych centoridów tworzymy prostokąt troche powiększony.
    height_min = np.min(data[:, 0])
    width_min = np.min(data[:, 1])
    height_max = np.max(data[:, 0])
    width_max = np.max(data[:, 1])
    
    img_height, img_width = img.shape
    
    start_point_height = int(max(height_min-img_height*0.1, 1))
    start_point_width = int(max(width_min-img_width*0.15, 1))
    end_point_height = int(height_max+img_height*0.1)
    end_point_width = int(width_max+img_width*0.25)
    
    # Wycięcie tego prostokątu z oryginalnego obrazu.
    img_org_path = Path('../data/ocr1') / str(image_path.stem + ".jpg")
    img_org = io.imread(img_org_path)
    img_org = color.rgb2gray(img_org)    
    img_slice = img_org[start_point_height:end_point_height, start_point_width:end_point_width]
#     io.imsave(arr=util.img_as_ubyte(img_slice), fname=save_path / (image_path.stem+'.png'))
    
    # Na wycinkach nadal czasami pojawia się stół. Więc usuwamy te fragmenty.

    p2, p98 = np.percentile(img_slice, (2, 98))
    img_slice = exposure.rescale_intensity(img_slice, in_range=(p2, p98))
        
    img_removed_background = remove_background_rectangle(img_slice)

    (save_path / "usuniety_stol").mkdir(parents=True, exist_ok=True)
    io.imsave(arr=util.img_as_ubyte(img_removed_background), fname=save_path / "usuniety_stol" / (image_path.stem+'.png'))

    
    #     break
    
    

    


..\data\partial_results\canny_adam\img_1.png
..\data\partial_results\canny_adam\img_10.png
..\data\partial_results\canny_adam\img_11.png
..\data\partial_results\canny_adam\img_12.png
..\data\partial_results\canny_adam\img_13.png
..\data\partial_results\canny_adam\img_14.png
..\data\partial_results\canny_adam\img_15.png
..\data\partial_results\canny_adam\img_16.png
..\data\partial_results\canny_adam\img_17.png
..\data\partial_results\canny_adam\img_18.png
..\data\partial_results\canny_adam\img_19.png
..\data\partial_results\canny_adam\img_2.png
..\data\partial_results\canny_adam\img_20.png
..\data\partial_results\canny_adam\img_21.png
..\data\partial_results\canny_adam\img_22.png
..\data\partial_results\canny_adam\img_23.png
..\data\partial_results\canny_adam\img_24.png
..\data\partial_results\canny_adam\img_25.png
..\data\partial_results\canny_adam\img_26.png
..\data\partial_results\canny_adam\img_27.png
..\data\partial_results\canny_adam\img_28.png
..\data\partial_results\canny_adam\i

# Wykrywanie linii poziomych - nieużywane

In [385]:
import cv2
from skimage.transform import probabilistic_hough_line
import matplotlib.pyplot as plt
from matplotlib import cm

def get_angle(line):
    angle = np.rad2deg(np.arctan2(line[1][1] - line[0][1], line[1][0] - line[0][0]))
    return angle
    
def hugh_and_rotation(image):
    kernelg = np.ones((3,3),np.uint8)
    gradient = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernelg)
    edges = cv2.Canny(gradient,200,300)
    edges = morphology.dilation(edges, morphology.square(3))

    
#     plt.gcf().set_size_inches(11, 7)
#     plt.imshow(gradient,cmap = 'gray'),plt.title('closing1')
#     plt.show()
#     plt.gcf().set_size_inches(11, 7)
#     plt.imshow(edges,cmap = 'gray'),plt.title('closing1')
#     plt.show() 
    
    
    angles = np.linspace((np.pi / 2)-0.10, (np.pi / 2)+0.10, 360)
#     angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
    lines = probabilistic_hough_line(edges, threshold=10, line_length=600,
                                     line_gap=25,theta=angles)

    # Generating figure 2
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].imshow(image, cmap=cm.gray)
    ax[0].set_title('Input image')

    ax[1].imshow(edges, cmap=cm.gray)
    ax[1].set_title('Canny edges')

    ax[2].imshow(edges * 0)
    for line in lines:
        p0, p1 = line
        ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
    ax[2].set_xlim((0, image.shape[1]))
    ax[2].set_ylim((image.shape[0], 0))
    ax[2].set_title('Probabilistic Hough')
    
    if len(lines) > 0:
        angle = get_angle(lines[0])
        image = rotate(image, angle)

    
    
    return image, lines
    
source_path = Path('../data/ocr1/img_29.jpg')

img = cv2.imread(str(source_path), cv2.IMREAD_GRAYSCALE)
p2, p98 = np.percentile(img, (2, 98))
img = exposure.rescale_intensity(img, in_range=(p2, p98))

img = global_thresholding_v2(img)

img = util.img_as_ubyte(img)*255
img = add_border_to_image(img)

image2, lines = hugh_and_rotation(img)

# plt.gcf().set_size_inches(11, 7)
# plt.imshow(img,cmap = 'gray'),plt.title('closing')
# plt.show() 

plt.gcf().set_size_inches(11, 7)
plt.imshow(image2,cmap = 'gray'),plt.title('closing')
plt.show() 

NameError: name 'rotate' is not defined

In [374]:
def add_border_to_image(image, width_of_border=30, intensity=0):
    (h, w) = image.shape
    new_image_left_border = np.full((h,w+width_of_border), fill_value=intensity, dtype=np.uint8)
    new_image_left_border[:,width_of_border:] = image
    image = new_image_left_border
    
    (h, w) = image.shape
    new_image_right_border = np.full((h,w+width_of_border), fill_value=intensity,  dtype=np.uint8)
    new_image_right_border[:,:-width_of_border] = image
    image = new_image_right_border
    
    (h, w) = image.shape
    new_image_bot_border = np.full((h+width_of_border,w), fill_value=intensity,  dtype=np.uint8)
    new_image_bot_border[:-width_of_border,:] = image
    image = new_image_bot_border
    
    (h, w) = image.shape
    new_image_top_border = np.full((h+width_of_border,w), fill_value=intensity,  dtype=np.uint8)
    new_image_top_border[width_of_border:,:] = image
    image = new_image_top_border
    return image

def local_thresholding(img):   
#     img = rgb2gray(img)
#     img = img_as_ubyte(img)
#     height, width = img.shape
#     temp = int((height+width)/10)
#     block_size = temp if temp%2==1 else temp+1
#     offset = np.mean(img.flatten())/2
    local_thresh = filters.threshold_local(img, 35, offset=0, method='gaussian')

    binary_local = img > local_thresh
    
    return binary_local

def global_thresholding(img):   
#     img = rgb2gray(img)
#     img = img_as_ubyte(img)

    global_thresh = filters.threshold_otsu(img)
    binary_global = img > global_thresh
    
    return binary_global  

def global_thresholding_v2(img, rows=2, columns=2):   
    height, width = img.shape
    
    height_part = int(height/rows)
    width_part = int(width/columns)
    
    for row in range(rows):
        for col in range(columns):
#             sector = img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part]
            img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part] = global_thresholding(img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part])
        
    return img   